# Log 01-10-2023

## TODOs

- [x] test other cases
- [x] list reliable gams file sources
    - its own library should be enough
- [ ] better handle exceptions

## Observations

### 14:57 compiler_variable
- `%gamsfolder%`
- `%modeltype%`

## Code

In [7]:
from gams2pyomo import GAMSTranslator

f = './test/gams_model_library/trnsport.gms'

gp = GAMSTranslator(f)

tree = gp.parse()
with open('tmp/tree', 'w') as file:
    file.write(tree.pretty())

res = gp.transform()
with open('tmp/debug.py', 'w') as file:
    file.write(res)

In [2]:
from pyomo.environ import *
import math


m = ConcreteModel()
options = {}


"""This problem finds a least cost shipping schedule that meets
requirements at markets and supplies at factories.


Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.
Princeton University Press, Princeton, New Jersey, 1963.

This formulation is described in detail in:
Rosenthal, R E, Chapter 2: A GAMS Tutorial. In GAMS: A User's Guide.
The Scientific Press, Redwood City, California, 1988.

The line numbers will not match those in the book because of these
comments.

Keywords: linear programming, transportation problem, scheduling
"""

m.I = Set(initialize=['seattle', 'san-diego'], ordered=True, doc='canning plants')
m.J = Set(initialize=['new-york', 'chicago', 'topeka'], ordered=True, doc='markets')
m.a = Param(m.I, mutable=True, initialize={'seattle': 350, 'san-diego': 600}, doc='capacity of plant i in cases')
m.b = Param(m.J, mutable=True, initialize={'new-york': 325, 'chicago': 300, 'topeka': 275}, doc='demand at market j in cases')
m.d = Param(m.I, m.J, mutable=True, initialize={('seattle', 'new-york'): 2.5, ('seattle', 'chicago'): 1.7, ('seattle', 'topeka'): 1.8, ('san-diego', 'new-york'): 2.5, ('san-diego', 'chicago'): 1.8, ('san-diego', 'topeka'): 1.4}, doc='distance in thousands of miles')
m.f = Param(mutable=True, initialize=90, doc='freight in dollars per case per thousand miles')
m.c = Param(m.I, m.J, mutable=True, doc='transport cost in thousands of dollars per case')
for i in m.I:
	for j in m.J:
		m.c[i, j] = (m.f * m.d[i, j]) / 1000
m.x = Var(m.I, m.J, doc='shipment quantities in cases')
m.z = Var(doc='total transportation costs in thousands of dollars')
m.x.domain = NonNegativeReals
def cost(m):
	return m.z == sum((m.c[i, j] * m.x[i, j]) for i in m.I for j in m.J)
m.cost = Constraint(rule=cost)
def supply(m, i):
	return sum(m.x[i, j] for j in m.J) <= m.a[i]
m.supply = Constraint(m.I, rule=supply)
def demand(m, j):
	return sum(m.x[i, j] for i in m.I) >= m.b[j]
m.demand = Constraint(m.J, rule=demand)
m._obj_ = Objective(rule=m.z, sense=1)
opt = SolverFactory('gurobi')
opt.solve(m, tee=True)
m.x.pprint()


Academic license - for non-commercial use only - expires 2023-12-11
Using license file /Users/pengfeicheng/gurobi.lic
Read LP format model from file /var/folders/02/59xf2rz134l9w97kw2rhkmch0000gn/T/tmpz3hmbmj5.pyomo.lp
Reading time = 0.00 seconds
x8: 7 rows, 8 columns, 20 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 8 columns and 20 nonzeros
Model fingerprint: 0xcbe7d830
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.125000e+02   0.000000e+00      0s
       3    1.5367500e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seco

In [ ]:
m.d

In [5]:
m.d.pprint()

d : distance in thousands of miles
    Size=4, Index=d_index, Domain=Any, Default=None, Mutable=True
    Key                       : Value
     ('san-diego', 'chicago') : 1.8
    ('san-diego', 'new-york') : 2.5
      ('san-diego', 'topeka') : 1.4
      ('seattle', 'new-york') : <class 'pyomo.core.base.param.Param.NoValue'>
